# Remote Downloader for HCP Aspera packages

This is a helper tool to download HCP aspera packages on systems which do not have browser or GUI access. You use your local machine (where you do have browser access, and aspera installed) to navigate to db.humanconnectome.org and initiate a transfer. When you do this, Aspera saves the transfer settings (including a token) to a local file. You can immediately cancel the transfer, then run this tool to initiate that same transfer on a remote system using Aspera CLI.

## Instructions:
- Step 1: In another browser window, use your local machine (i.e. where you do have browser access, and aspera installed, and running this notebook) to navigate to db.humanconnectome.org and initiate a transfer.
- Step 2: Cancel that transfer before it begins to actually download the files.
- Step 3: Run the cells in this notebook, which will read the Aspera transfers db (sql) and pull out the info, including download tokens
- Step 4: You can select the transfer you want to initiate, and a Aspera CLI command will be generated that can initiate the download on a remote system where Aspera CLI is installed.


## Dependencies:

Install Aspera Connect (includes Aspera CLI) on linux (for remote system):
```
#install aspera connect (installs to ~/.aspera):
wget https://download.asperasoft.com/download/sw/connect/3.9.9/ibm-aspera-connect-3.9.9.177872-linux-g2.12-64.tar.gz
tar -xvzf ibm-aspera-connect-3.9.9.177872-linux-g2.12-64.tar.gz
./ibm-aspera-connect-3.9.9.177872-linux-g2.12-64.sh
```

Python modules:
- sqlite3
- json
- subprocess
- os


In [12]:
from sqlite3 import connect
from json import loads
from subprocess import run
from os.path import expanduser,join

home = expanduser("~")


#on machine where connectomedb browser transfer was initiated
conn = connect(join(home,'.aspera/connect/var/asperaconnect.data'))


##on destination system; if mac os X:
#destination = join(home,'incoming_hcp')
#key = join(home,'Applications/Aspera\ Connect.app/Contents/Resources/asperaweb_id_dsa.openssh')
#ascp = join(home,'Applications/Aspera\ Connect.app/Contents/Resources/ascp')


##destination system; if linux:
#remotehome = '/home/akhanf'
#destination = join(remotehome,'incoming_hcp')
#key = join(remotehome,'.aspera/connect/etc/asperaweb_id_dsa.openssh')
#ascp = join(remotehome,'.aspera/cli/bin/ascp')

In [13]:
#get entries in local aspera transfer database

c = conn.cursor()
c.execute('SELECT data FROM transfer ORDER BY data DESC')
rows = c.fetchall()

transfer_dicts = [loads(*rows[i]) for i in range(len(rows))]

first_file_list = [transfer_dicts[i]['paths'][0]['source'].split('/')[-1]  for i in range(len(rows))]
token_list = [transfer_dicts[i]['token']  for i in range(len(rows))]
first_file_list
nfiles_list = [len(transfer_dicts[i]['paths'])  for i in range(len(rows))]

#make dropdown list options as the first file, and the number of files:
dropdown_options = [ 'first file: {}, total files = {}'.format(first_file_list[i],nfiles_list[i]) for i in range(len(rows))]

# display a widget
import ipywidgets as widgets

w = widgets.Dropdown(
    options=dropdown_options,
    description='Download:',
    disabled=False,
    layout={'width': 'max-content'}
)
display(w)

Dropdown(description='Download:', layout=Layout(width='max-content'), options=('first file: 100610_7T_RET_2mm_…

In [14]:
# generate command for local download -- testing only..

#on destination system; if mac os X:
destination = join(home,'incoming_hcp')
key = join(home,'Applications/Aspera\ Connect.app/Contents/Resources/asperaweb_id_dsa.openssh')
ascp = join(home,'Applications/Aspera\ Connect.app/Contents/Resources/ascp')

transfer_dict = transfer_dicts[w.index]

path_list = [path['source'] for path in transfer_dict['paths'] ]
path_string = ' '.join(path_list)


cmdline_opts = '-c {encryption_cipher} --mode recv -O {udp_port} -y 1 -t 443 --host={host}'\
               ' --user={host_user} -i {key} -W {token} -P {host_port} {path_string} {destination}'.format(key=key,path_string=path_string,destination=destination,**transfer_dict)


#print the Aspera CLI that can be used on this local system (e.g. on your Mac) - useful for testing.. 
print(f'{ascp} {cmdline_opts}')

/Users/alik/Applications/Aspera\ Connect.app/Contents/Resources/ascp -c aes-128 --mode recv -O 33001 -y 1 -t 443 --host=asp-connect1.wustl.edu --user=asperaxfer -i /Users/alik/Applications/Aspera\ Connect.app/Contents/Resources/asperaweb_id_dsa.openssh -W ATM2_ACsFqKR2pN3M7wlPihaS4TjnwM-qW24vIKLfkayR-WerYQAW4kr8KKp5XFfLw1Y7Rtzr5X_2MTA -P 33001 packages/live/HCP_1200/100610/fix/100610_7T_RET_2mm_fix.zip packages/live/HCP_1200/100610/fix/100610_7T_RET_2mm_fix.zip.md5 packages/live/HCP_1200/102311/fix/102311_7T_RET_2mm_fix.zip packages/live/HCP_1200/102311/fix/102311_7T_RET_2mm_fix.zip.md5 packages/live/HCP_1200/102816/fix/102816_7T_RET_2mm_fix.zip packages/live/HCP_1200/102816/fix/102816_7T_RET_2mm_fix.zip.md5 packages/live/HCP_1200/104416/fix/104416_7T_RET_2mm_fix.zip packages/live/HCP_1200/104416/fix/104416_7T_RET_2mm_fix.zip.md5 packages/live/HCP_1200/105923/fix/105923_7T_RET_2mm_fix.zip packages/live/HCP_1200/105923/fix/105923_7T_RET_2mm_fix.zip.md5 packages/live/HCP_1200/108323/fix/

In [17]:
# generate command for linux download:

transfer_dict = transfer_dicts[w.index]

remotehome = '~'
destination = join(remotehome,'incoming_hcp')
key = join(remotehome,'.aspera/connect/etc/asperaweb_id_dsa.openssh')
ascp = join(remotehome,'.aspera/cli/bin/ascp')


path_list = [path['source'] for path in transfer_dict['paths'] ]
path_string = ' '.join(path_list)


cmdline_opts = '-c {encryption_cipher} --mode recv -O {udp_port} -y 1 -t 443 --host={host}'\
               ' --user={host_user} -i {key} -W {token} -P {host_port} --overwrite=never -k 1 --policy=high {path_string} {destination}'.format(key=key,path_string=path_string,destination=destination,**transfer_dict)

#cmd = f'{cmdline_opts} {path_list} {destination}'
print(f'{ascp} {cmdline_opts}')

~/.aspera/cli/bin/ascp -c aes-128 --mode recv -O 33001 -y 1 -t 443 --host=asp-connect1.wustl.edu --user=asperaxfer -i ~/.aspera/connect/etc/asperaweb_id_dsa.openssh -W ATM2_ACsFqKR2pN3M7wlPihaS4TjnwM-qW24vIKLfkayR-WerYQAW4kr8KKp5XFfLw1Y7Rtzr5X_2MTA -P 33001 --overwrite=never -k 1 --policy=high packages/live/HCP_1200/100610/fix/100610_7T_RET_2mm_fix.zip packages/live/HCP_1200/100610/fix/100610_7T_RET_2mm_fix.zip.md5 packages/live/HCP_1200/102311/fix/102311_7T_RET_2mm_fix.zip packages/live/HCP_1200/102311/fix/102311_7T_RET_2mm_fix.zip.md5 packages/live/HCP_1200/102816/fix/102816_7T_RET_2mm_fix.zip packages/live/HCP_1200/102816/fix/102816_7T_RET_2mm_fix.zip.md5 packages/live/HCP_1200/104416/fix/104416_7T_RET_2mm_fix.zip packages/live/HCP_1200/104416/fix/104416_7T_RET_2mm_fix.zip.md5 packages/live/HCP_1200/105923/fix/105923_7T_RET_2mm_fix.zip packages/live/HCP_1200/105923/fix/105923_7T_RET_2mm_fix.zip.md5 packages/live/HCP_1200/108323/fix/108323_7T_RET_2mm_fix.zip packages/live/HCP_1200/10

In [28]:
#run command remotely:

# to do


'test_file'

In [90]:
p = run([ascp,cmdline_opts])
output = p.stdout
p

CompletedProcess(args=['/Users/alik/Applications/Aspera Connect.app/Contents/Resources/ascp', "-c aes-128 --mode recv -O 33001 -y 1 -t 443 --host=asp-connect1.wustl.edu --user=asperaxfer -i /Users/alik/Applications/Aspera Connect.app/Contents/Resources/aspera_web_key.pem -W ATM2_ACsf_5SngLPUy508rBUFIBo_wf4TFvO0LAl1sxiZwbz9-cAAI4AqIsQpq8008_Al9B5AMm_2MTA -P 33001 ['packages/live/HCP_1200/100307/preproc/100307_3T_tfMRI_SOCIAL_preproc.zip', 'packages/live/HCP_1200/100307/preproc/100307_3T_tfMRI_SOCIAL_preproc.zip.md5'] /Users/alik/incoming_hcp"], returncode=1)

In [74]:
print(out)

CompletedProcess(args=['/Users/alik/Applications/Aspera Connect.app/Contents/Resources/ascp', "-c aes-128 --mode recv -O 33001 -y 1 -t 443 --host=asp-connect1.wustl.edu --user=asperaxfer -i /Users/alik/Applications/Aspera Connect.app/Contents/Resources/aspera_web_key.pem -W ATM2_ACsf_5SngLPUy508rBUFIBo_wf4TFvO0LAl1sxiZwbz9-cAAI4AqIsQpq8008_Al9B5AMm_2MTA -P 33001 ['packages/live/HCP_1200/100307/preproc/100307_3T_tfMRI_SOCIAL_preproc.zip', 'packages/live/HCP_1200/100307/preproc/100307_3T_tfMRI_SOCIAL_preproc.zip.md5'] /Users/alik/incoming_hcp"], returncode=1)


In [95]:
print(path_list)

['packages/live/HCP_1200/100307/preproc/100307_3T_tfMRI_SOCIAL_preproc.zip', 'packages/live/HCP_1200/100307/preproc/100307_3T_tfMRI_SOCIAL_preproc.zip.md5']


'packages/live/HCP_1200/100307/preproc/100307_3T_tfMRI_SOCIAL_preproc.zip packages/live/HCP_1200/100307/preproc/100307_3T_tfMRI_SOCIAL_preproc.zip.md5'

In [104]:
print(transfer_dict)

{'account_id': '', 'added_time': 1589466362205, 'allow_dialogs': True, 'app_id': 'NzAwNzY0MDctYzI5Yy00Y2VjLWIyY2QtMDQ2ZmZjYTk4YjZk', 'authentication_method': 'public_key', 'auto_detect_path_mtu': True, 'auto_start': False, 'back_link': 'https://db.humanconnectome.org/app/action/ChooseDownloadPackages', 'bandwidth_cap': 0, 'bit_rate': 0, 'bytes_expected': 0, 'bytes_written': 0, 'content_protection': False, 'content_protection_optional': False, 'cookie': 'XDATUser=akhanf', 'create_path': False, 'current_file': '', 'data_folder': '/Users/alik/.aspera/connect/filelists/ibm aspera connect-6e1c4d13-f80a-4d14-810e-58925417f5fb', 'datagram_size': 1492, 'delete_upload_files_on_complete': False, 'destination_root': '/Users/alik/Downloads', 'encryption_cipher': 'aes-128', 'end_time': 0, 'error_code': 0, 'error_message': '', 'fasp_session_id': '561211f0-5ac2-4317-ba91-364aa10768b3', 'host': 'asp-connect1.wustl.edu', 'host_port': 33001, 'host_user': 'asperaxfer', 'http_fallback_allowed': True, 'htt